In [5]:
import pandas as pd
import numpy as np

data_train = pd.read_csv('../input/train.csv')
X = data_train.drop(['ID', 'medv'], axis=1)
y = data_train['medv'].values

In [6]:
# 去掉离群点
X = X[y != 50]
y = y[y != 50]

In [7]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(nthread=7)

In [9]:
from sklearn.model_selection import GridSearchCV

grid_params = dict(
    max_depth=[4, 5, 6, 7],
    learning_rate=np.linspace(0.03, 0.3, 10),
    n_estimators=[100, 200])
grid = GridSearchCV(
    xgb_model,
    grid_params,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=1)

In [10]:
%%time
grid.fit(X, y)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


CPU times: user 23.9 s, sys: 37 ms, total: 23.9 s
Wall time: 23.9 s


[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   23.8s finished
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=7, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [4, 5, 6, 7], 'learning_rate': array([0.03, 0.06, 0.09, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27, 0.3 ]), 'n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [11]:
print(grid.best_params_)

{'learning_rate': 0.09, 'max_depth': 5, 'n_estimators': 200}


In [13]:
print('rmse:', (-grid.best_score_)**0.5)

rmse: 4.027570642619267


使用全部的训练数据集重新训练一次。

In [14]:
xgbr_best = XGBRegressor(nthread=7, **grid.best_params_)
xgbr_best.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.09, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=7, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
X_test_pred = pd.read_csv('../input/test.csv')
y_pred = xgbr_best.predict(X_test_pred.drop(['ID'],axis=1))
save_df = pd.DataFrame({'ID': X_test_pred.ID, 'medv': y_pred})
save_df.to_csv('../output/xgbr_boston_submission2.csv', index=False)
# 3.18793